## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Portofino Tower Condo\r300 S Pointe Dr 3801\rM...,3/27/2023,2,"$7,400,000.00",4510,"$1,640.80",Jorge Martinez,One Sotheby's International Re,Jorge Martinez,One Sotheby's International Re,Portofino Tower Condo 300 S Pointe Dr 3801 Mia...,Portofino Tower Condo,300 S Pointe Dr 3801 Miami Beach,"(25.7677055, -80.1348768)",25.767706,-80.134877
2,Oceania Bal Harbour\r10201 Collins Ave 501\rBa...,3/27/2023,59,"$7,100,000.00",2148,"$3,305.40",Jonathan Mann,Coldwell Banker Realty,Elizabeth Dougherty,"Compass Florida, LLC",Oceania Bal Harbour 10201 Collins Ave 501 Bal ...,Oceania Bal Harbour,10201 Collins Ave 501 Bal Harbour,"(25.8951514, -80.1230855)",25.895151,-80.123086
3,Continuum on South Beach\r100 S Pointe Dr 3302...,3/31/2023,168,"$5,995,000.00",1791,"$3,347.29",Colin Feuling,Brown Harris Stevens,Carlo Dipasquale,"Compass Florida, LLC",Continuum on South Beach 100 S Pointe Dr 3302 ...,Continuum on South Beach,100 S Pointe Dr 3302 Miami Beach,"(25.7672315, -80.1327611)",25.767232,-80.132761
4,Two Park Grove Condo\r2821 S Bayshore Dr LPH-D...,3/31/2023,93,"$5,000,000.00",2917,"$1,714.09",Raul Santidrian PA,Douglas Elliman,Annie De La Rosa,Douglas Elliman,Two Park Grove Condo 2821 S Bayshore Dr LPH-D ...,Two Park Grove Condo,2821 S Bayshore Dr LPH-D Miami,"(25.7287205, -80.23808009999999)",25.728721,-80.238080
5,Villas at Ritz Carlton\r820 W 47th Ct\rMiami B...,3/27/2023,1,"$4,975,000.00",3290,"$1,512.16",Comp Sale,Comp Only Sales,Oliver Lloyd,Douglas Elliman,Villas at Ritz Carlton 820 W 47th Ct Miami Beach,Villas at Ritz Carlton,820 W 47th Ct Miami Beach,"(25.8227346, -80.131993)",25.822735,-80.131993
6,The Bath Club\r5959 Collins Ave 1802\rMiami Beach,3/29/2023,139,"$4,650,000.00",2572,"$1,807.93",Patty Marun,Suma Luxury Realty,Tara Elias Schuchts,Douglas Elliman,The Bath Club 5959 Collins Ave 1802 Miami Beach,The Bath Club,5959 Collins Ave 1802 Miami Beach,"(25.8422691, -80.1203074)",25.842269,-80.120307
7,Bellini Condo\r10225 Collins Ave 2101\rBal Har...,3/30/2023,127,"$4,500,000.00",3820,"$1,178.01",David Cohen,The Corcoran Group,Fernanda Zomignani PA,"AG Real Estate Advisors, LLC",Bellini Condo 10225 Collins Ave 2101 Bal Harbour,Bellini Condo,10225 Collins Ave 2101 Bal Harbour,"(25.8964898, -80.12326499999999)",25.896490,-80.123265
8,Muse\r17141 Collins Ave 2001\rSunny Isles Beach,3/31/2023,52,"$4,450,000.00",3195,"$1,392.80",Tania Dalaneze,Dalaneze International Realty,Maria Kafetzis,Douglas Elliman,Muse 17141 Collins Ave 2001 Sunny Isles Beach,Muse,17141 Collins Ave 2001 Sunny Isles Beach,"(25.9349392, -80.12091819999999)",25.934939,-80.120918
9,Fisher Island\r5232 Fisher Island Dr 5232\rMia...,3/29/2023,1,"$4,250,000.00",1950,"$2,179.49",Lazaro Rivero,"Rivero Real Estate , Inc.",Lazaro Rivero,"Rivero Real Estate , Inc.",Fisher Island 5232 Fisher Island Dr 5232 Miami...,Fisher Island,5232 Fisher Island Dr 5232 Miami Beach,"(25.7626891, -80.1484152)",25.762689,-80.148415
10,0St Regis Bal Harbour\r9701 Collins Ave 604S\r...,3/28/2023,33,"$4,250,000.00",2505,"$2,223.97",Fernanda Zomignani PA,"AG Real Estate Advisors, LLC",Barbara Frampton,"M. Kotler Realty, Inc.",0St Regis Bal Harbour 9701 Collins Ave 604S Ba...,,0St Regis Bal Harbour 9701 Collins Ave 604S Ba...,"(25.8884913, -80.1228812)",25.888491,-80.122881


In [9]:
df.at[10,'building_name']=('St Regis Bal Harbour')
df.at[10,'address_only']=('9701 Collins Ave')

## Format Data

In [10]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [11]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=True)
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [12]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [13]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [14]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['price_per_sqft']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [15]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('March 27th - April 1st')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [16]:
m.save('index.html')

## Data snagger

In [17]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [18]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Portofino Tower Condo , 300 S Pointe Dr 3801 Miami Beach | Price $7,400,000 | $1,641 psf | Listing agent: Jorge Martinez with One Sotheby's International Re | Buyer's agent: Jorge Martinez with One Sotheby's International Re | Days on market: 2
Least Expensive
Fisher Island , 5232 Fisher Island Dr 5232 Miami Beach | Price $4,250,000 | $2,179 psf | Listing agent: Lazaro Rivero with Rivero Real Estate , Inc. | Buyer's agent: Lazaro Rivero with Rivero Real Estate , Inc. | Days on market: 1
Highest Price Per Square Foot
Continuum on South Beach , 100 S Pointe Dr 3302 Miami Beach | Price $5,995,000 | $3,347 psf | Listing agent: Colin Feuling with Brown Harris Stevens | Buyer's agent: Carlo Dipasquale with Compass Florida, LLC | Days on market: 168
Lowest Price Per Square Foot
Bellini Condo , 10225 Collins Ave 2101 Bal Harbour | Price $4,500,000 | $1,178 psf | Listing agent: David Cohen with The Corcoran Group | Buyer's agent: Fernanda Zomignani PA with AG Real Estate Advisors

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [19]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [20]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_04022023
